# Proyecto C19 Python

In [3]:
#Imports
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import norm
from scipy import stats
import os
import glob

In [62]:
#Define dataset de examenes PCR por region/fecha
pcrStdDf = pd.read_csv(os.getcwd() + "/dataSets/producto7/PCR_std.csv", index_col=None, header=0)
pcrStdDf.columns = ['region', 'codigo_region', 'poblacion', 'fecha', 'numero_pcr']

#Define dataset de casos nuevos por region/fecha
casosNuevosDf = pd.read_csv(os.getcwd() + "/dataSets/producto13/CasosNuevosCumulativo_std.csv", index_col=None, header=0)
casosNuevosDf.columns = ['region', 'fecha', 'total']
#ToDo verificar el tema del codigo de region para casosNuevosDf
#https://www.dataquest.io/blog/tutorial-add-column-pandas-dataframe-based-on-if-else-condition/

In [73]:
casosNuevosDfCondicionesCodRegion = (
    (casosNuevosDf['region'] == 'Arica y Parinacota'),
    (casosNuevosDf['region'] == 'Tarapacá'),
    (casosNuevosDf['region'] == 'Antofagasta'),
    (casosNuevosDf['region'] == 'Atacama'),
    (casosNuevosDf['region'] == 'Coquimbo'),
    (casosNuevosDf['region'] == 'Valparaíso'),
    (casosNuevosDf['region'] == 'Metropolitana'),
    (casosNuevosDf['region'] == 'O’Higgins'),
    (casosNuevosDf['region'] == 'Maule'),
    (casosNuevosDf['region'] == 'Ñuble'),
    (casosNuevosDf['region'] == 'Biobío'),
    (casosNuevosDf['region'] == 'Araucanía'),
    (casosNuevosDf['region'] == 'Los Ríos'),
    (casosNuevosDf['region'] == 'Los Lagos'),
    (casosNuevosDf['region'] == 'Aysén'),
    (casosNuevosDf['region'] == 'Magallanes')
)

casosNuevosDfValoresCodRegion = [
    '1',
    '2',
    '3',
    '4',
    '5',
    '6',
    '7',
    '8',
    '9',
    '10',
    '11',
    '12',
    '13',
    '14',
    '15',
    '16'
]

casosNuevosDf['codigo_region'] = np.select(casosNuevosDfCondicionesCodRegion, casosNuevosDfValoresCodRegion)

casosNuevosDf.head(18)

,region,fecha,total,codigo_region
0,Arica y Parinacota,2020-03-03,0.0,1
1,Tarapacá,2020-03-03,0.0,2
2,Antofagasta,2020-03-03,0.0,3
3,Atacama,2020-03-03,0.0,4
4,Coquimbo,2020-03-03,0.0,5
5,Valparaíso,2020-03-03,0.0,6
6,Metropolitana,2020-03-03,0.0,7
7,O’Higgins,2020-03-03,0.0,8
8,Maule,2020-03-03,0.0,9
9,Ñuble,2020-03-03,0.0,10


In [ ]:
#Define dataframes product2/4 en un mapa, y cada key es una fecha de un archivo encontrado
def loadDataFrameToMap(folder, indexKey) :
        
    files = [os.path.basename(x) for x in glob.glob(folder)]
    
    mapDataFramesByDate = {}

    for fileName in files:
        keyMap = int((fileName[0: -indexKey]).replace("-", ''))
        mapDataFramesByDate[keyMap] = pd.read_csv(folder[0: -5] + fileName, index_col=None, header=0)
    
    #ToDo: ordenar por orden ascendente
    return mapDataFramesByDate

In [ ]:
pathProduct2 = os.getcwd() + "/dataSets/producto2/*.csv";
product2MapDF = loadDataFrameToMap(pathProduct2, 21)
#product2MapDF[20210201]

pathProduct4 = os.getcwd() + "/dataSets/producto4/*.csv";
product4MapDF = loadDataFrameToMap(pathProduct4, 35)
#product4MapDF[20210607]

In [ ]:
#ToDo: verificar si es necesaria o no esta funcion!!!!
def obtainDataFrames(mapDataFrames, fromDate, toDate, exclusiveFrom, exclusiveTo) :
    
    if (exclusiveFrom) :
        fromDate = fromDate + 1
        
    if (exclusiveTo) :
        toDate = toDate - 1
    
    #print(fromDate, toDate)
    
    mapBetween = {}
    for mapKey in mapDataFrames.keys() :
        if(mapKey > fromDate and mapKey < toDate) :
            mapBetween[mapKey] = mapDataFrames[mapKey]
        
    return mapBetween


#obtainDataFrames(product4MapDF, 20210101, 20210110, True, True)

# Positividad de casos

(Proporcion de test positivos por dia / Total de test efectuados en un dia) * 100

- Proporcion de test positivos por dia => DataSet product4 col: "casos nuevos totales"
- Total de test efectuados en un dia => DataSet product7 col: "numero"

In [58]:
def obtenerPcrPorFechaYDistrito(fecha, distrito) :
    predicadoFecha = pcrStdDf["fecha"] == fecha
    predicadoDistrito = pcrStdDf["codigo_region"] == distrito
    
    return pcrStdDf[
        (predicadoDistrito) & 
        (predicadoFecha)
    ]

obtenerPcrPorFechaYDistrito('2021-08-04', 5)

          region  codigo_region  poblacion       fecha  numero_pcr
7717  Valparaíso              5    1960170  2021-08-04      2062.0


In [77]:
def obtenerCasosNuevosPorFechaYDistrito(fecha, distrito) :
    predicadoFecha = casosNuevosDf["fecha"] == fecha
    predicadoDistrito = casosNuevosDf["codigo_region"] == distrito
    
    return casosNuevosDf[
        (predicadoFecha) 
    ]

obtenerCasosNuevosPorFechaYDistrito('2021-08-09', 8)

,region,fecha,total,codigo_region
8908,Arica y Parinacota,2021-08-09,23.0,1
8909,Tarapacá,2021-08-09,23.0,2
8910,Antofagasta,2021-08-09,17.0,3
8911,Atacama,2021-08-09,15.0,4
8912,Coquimbo,2021-08-09,25.0,5
8913,Valparaíso,2021-08-09,65.0,6
8914,Metropolitana,2021-08-09,437.0,7
8915,O’Higgins,2021-08-09,25.0,8
8916,Maule,2021-08-09,53.0,9
8917,Ñuble,2021-08-09,9.0,10


In [35]:
def calcularPositividad(cantidadPcrDiario, casosPositivosDiarios) :
    positividad = ((cantidadPcrDiario / casosPositivosDiarios) * 100)

In [ ]:
def xyz(dataFame, fecha, distrito) :
    
    #ToDo: sacar los '-' a variable fecha
    dataFame[fecha][]
    